### Geocoding Toronto Neighborhoods

First I install the needed packages. Beautifulsoup4 and lxml for webscrapping and geocoder for, well, geocoding.

In [1]:
import sys
!{sys.executable} -m pip install beautifulsoup4
!{sys.executable} -m pip install lxml
!{sys.executable} -m pip install geocoder

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import geocoder

In [3]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, "lxml")
html_table = soup.table

In [4]:
postal_codes = pd.read_html(str(html_table))[0]
postal_codes = postal_codes.loc[postal_codes.Borough != "Not assigned"]
postal_list = postal_codes.loc[:,"Postcode"].drop_duplicates().to_list()

I request the data from wikipedia and take the first table found in the page, check that it's indeed the Postcode data set, drop missing values and make a list with all the unique values of the postcodes.

In [5]:
list_of_df = list()
for postal_code in postal_list:
    df = postal_codes.loc[postal_codes.Postcode == postal_code]
    neighborhoods = ', '.join(df.loc[:,"Neighbourhood"].to_list())
    df_2 = df.loc[:,["Postcode","Borough"]].drop_duplicates()
    df_2["Neighborhoods"] = neighborhoods
    list_of_df.append(df_2)
postal_codes = pd.concat(list_of_df)

To retrieve the data in the desired manner, I loop through the postcode list. For each value, I subset the original dataframe to get the name of the Borouh and a list of nieghborhoods, and then convert this list of neoghborhoods to a comma separated string and then assign it to the Neighborhood column. After doing this to everty postode, I concat all the data into a single data frame.

In [6]:
postal_codes.sample(20)

,Postcode,Borough,Neighborhoods
114,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel"
3,M4A,North York,Victoria Village
2,M3A,North York,Parkwoods
109,M2L,North York,"Silver Hills, York Mills"
76,M2J,North York,"Fairview, Henry Farm, Oriole"
155,M4P,Central Toronto,Davisville North
32,M4C,East York,Woodbine Heights
93,M2K,North York,Bayview Village
125,M2M,North York,"Newtonbrook, Willowdale"
248,M4X,Downtown Toronto,"Cabbagetown, St. James Town"


In [20]:
postal_codes.shape

(103, 5)